In [1]:
import numpy as np

In [2]:
def mpdist(
    x: np.ndarray,
    y: np.ndarray,
    m: int = 0
) -> float:
    """
    Matrix Profile Distance
    
    Parameters
    ----------
    x : np.ndarray
        First time series, univariate, shape ``(n_timepoints,)``
    y : np.ndarray
        Second time series, univariate, shape ``(n_timepoints,)``
    m : int (default = 0)
        Length of the subsequence

    Returns
    -------
    float
        Matrix Profile distance between x and y

    Raises
    ------
    ValueError
        If x and y are not 1D arrays
    """
    if x.ndim != 1 or y.ndim != 1:
        raise ValueError("x and y must be a 1D array of shape (n_timepoints,)")

    return _mpdist(x,y,m)
    

def stomp_ab(
    x: np.ndarray,
    y: np.ndarray,
    m: int = 0
) -> (np.array, np.array):
    """
    STOMP implementation for AB similarity join.
    
    Parameters
    ----------
    x : np.ndarray
        First time series, univariate, shape ``(n_timepoints,)``
    y : np.ndarray
        Second time series, univariate, shape ``(n_timepoints,)``
    m : int (default = 0)
        Length of the subsequence

    Output
    ------
        mp: np.ndarray
            Array with the distance between every subsequence from x
            to the nearest subsequence with same length from y.
        ip: np.ndaaray
            Array with the index of the nearest neighbor of x in y.
    """
    if x.ndim != 1 or y.ndim != 1:
        raise ValueError("x and y must be a 1D array of shape (n_timepoints,)")

    return _stomp_ab(x,y,m)


In [3]:
def _sliding_dot_products(
    q: np.ndarray,
    t: np.ndarray
)-> np.ndarray:
    """
    Compute the sliding dot products between a query and a time series.

    Parameters
    ----------
        q: numpy.array
            Query.
        t: numpy.array
            Time series.

    Output
    ------
        dot_prod: numpy.array
            Sliding dot products between q and t.
    """
    q_len = len(q)
    t_len = len(t)

    # Reversing query and padding both query and time series
    t_padded = np.pad(t, (0, t_len))
    q_reversed = np.flipud(q)
    q_reversed_padded = np.pad(q_reversed, (0, 2 * t_len - q_len))
    
    # Applying FFT to both query and time series
    t_fft = np.fft.fft(t_padded)
    q_fft = np.fft.fft(q_reversed_padded)  # Flip the query and FFT it

    # Applying inverse FFT to obtain the convolution of the time series by
    # the query
    element_wise_mult = np.multiply(t_fft, q_fft)
    inverse_fft = np.fft.ifft(element_wise_mult)

    # Returns only the valid dot products from inverse_fft
    dot_prod = inverse_fft[q_len - 1 : t_len].real

    return dot_prod

In [ ]:
# To be done

def _mpdist(x,y,m):
    threshold = 0.05

    # Compute the AB matrix profile
    mp_ab, ip_ab = _stomp_ab(x,y,m)

    # Compute the BA matrix profile
    mp_ba, ip_ba = _stomp_ab(y,x,m)

In [ ]:
# To be continues

def _stomp_ab(x,y,m):
    if m == 0:
        if len(x) > len(y):
            m = int(len(x)/4)
        else:
            m = int(len(y)/4)

    # Number of subsequences
    n_x_subs = len(x) - m + 1
    n_y_subs = len(y) - m + 1

    # Compute the mean and standard deviation
    x_mean = []
    x_std = []
    y_mean = []
    y_std = []

    for i in range(0, n_x_subs):
        x_subset = x[i:i+m]    
        x_mean.append(np.mean(x_subset))
        x_std.append(np.std(x_subset))

    for i in range(0, n_y_subs):
        y_subset = y[i:i+m]
        y_mean.append(np.mean(y_subset))
        y_std.append(np.std(y_subset))

    # Compute the dot products between the first y subsequence and every x subsequence
    dot_prod = _sliding_dot_products(y[0:m], x)
    first_dot_prod = np.copy(dot_prod)

### Fourier Transform

In [1]:
import numpy as np

a = np.array([5, 4, 6, 3, 7]) 
# using np.fft() method 
fourier_a = np.fft.fft(a) 
  
print(fourier_a)

[25.        +0.j          1.11803399+1.08981379j -1.11803399+4.61652531j
 -1.11803399-4.61652531j  1.11803399-1.08981379j]


In [5]:
ifourier_a = np.fft.ifft(fourier_a)
print(ifourier_a)

[5.+0.j 4.+0.j 6.+0.j 3.+0.j 7.+0.j]
